In [1]:
import cv2
import numpy as np
import os
import SimpleITK as sitk

Convert the labels into binary image

In [ ]:
rgb_folder_path = 'registration/rgb'
binary_folder_path = 'registration/binary'
files = os.listdir(rgb_folder_path)

# Loop through each file
for file_name in files:
    file_path = os.path.join(rgb_folder_path, file_name)
    if file_name.lower().endswith(('.png')):
        img = cv2.imread(file_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, binary_image = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)

        # Save the binary image with a new name
        binary_file_name = f'binary_{file_name}'
        binary_file_path = os.path.join(binary_folder_path, binary_file_name)
        cv2.imwrite(binary_file_path, binary_image)

        print(f'Processed: {file_name}, Binary image saved as: {binary_file_name}')

Registration

In [ ]:
parameter_map_affine = sitk.ReadParameterFile('registration/parameters/Affine_bin.txt')
moving_img = sitk.ReadImage('registration/binary/binary_atlas_six_labels.png')
binary_folder_path = 'registration/binary'
binary_files = os.listdir(binary_folder_path)

# Loop through each file
for file_name in binary_files:
    file_path = os.path.join(binary_folder_path, file_name)
    if file_name.lower().endswith(('.png')) and "binary_atlas_label" not in file_name.lower() and "binary_atlas_six_labels" not in file_name.lower():
        fixed_img = sitk.ReadImage(file_path)
        elastixImageFilter = sitk.ElastixImageFilter()
        elastixImageFilter.SetFixedImage(fixed_img)
        elastixImageFilter.SetMovingImage(moving_img)
        elastixImageFilter.SetOutputDirectory('registration/registration_output')

        # Affine registration
        elastixImageFilter.SetParameterMap(parameter_map_affine)
        elastixImageFilter.Execute()
        resultImage_affine = elastixImageFilter.GetResultImage()
        transformParameterMap_affine = elastixImageFilter.GetTransformParameterMap()[0]

        # Cast the pixel types to unsigned char
        resultImage_affine = sitk.Cast(resultImage_affine, sitk.sitkUInt8)

        parts = file_name.split("_")
        affine_registered_image_name = f"{parts[1]}_{parts[2]}_affine_reg.png"
        affine_transform_parameter_name = f"{parts[1]}_{parts[2]}_transform_affine.txt"

        # Save results for affine registration
        sitk.WriteImage(resultImage_affine, os.path.join('registration/moving_registered', affine_registered_image_name))
        sitk.WriteParameterFile(transformParameterMap_affine, os.path.join('registration/transform_parameters', affine_transform_parameter_name))

Overlay the registered label on the image

In [ ]:
binary_folder_path = 'registration/moving_registered'
binary_files = os.listdir(binary_folder_path)
gray_folder_path = 'registration/grayscale'
overlay_folder_path = 'registration/overlayed_thorax'

# Loop through each file
for file_name in binary_files:
    file_path = os.path.join(binary_folder_path, file_name)
    if file_name.lower().endswith(('.png')):
        parts = file_name.split("_")
        grayscale_image_path = f"{gray_folder_path}/{parts[0]}_{parts[1]}.png"

        # Read images
        gray_image = cv2.imread(grayscale_image_path)
        binary_mask = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        mask_rgba = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGRA)

        # Set alpha channel values
        alpha_factor = 0.3
        mask_rgba[:, :, 3] = binary_mask * alpha_factor

        # Convert grayscale image to have 3 channels (RGB)
        gray_image_rgb = cv2.cvtColor(gray_image, cv2.COLOR_BGR2BGRA)

        # Overlay the binary mask region on the grayscale image
        result = cv2.addWeighted(gray_image_rgb, 1, mask_rgba, 0.5, 0)

        # Save the overlayed image with a new name
        overlay_file_name = f"{parts[0]}_{parts[1]}_overlayed_thorax.png"
        overlay_file_path = os.path.join(overlay_folder_path, overlay_file_name)
        cv2.imwrite(overlay_file_path, result)

        print(f'Processed: {file_name}, overlayed image saved as: {overlay_file_name}')

Transform lung lobes

In [ ]:
pulmonary_regions_atlas_path = 'registration/rgb/atlas_six_labels.png'
moving_image = sitk.ReadImage(pulmonary_regions_atlas_path, sitk.sitkFloat32)
transform_parameters_path = 'registration/transform_parameters'
transform_parameter_files = os.listdir(transform_parameters_path)

# Loop through each transform parameter file
for file_name in transform_parameter_files:
    file_path = os.path.join(transform_parameters_path, file_name)
    if file_name.lower().endswith(('.txt')):
        transformParameterMap = sitk.ReadParameterFile(file_path)
        transformixImageFilter = sitk.TransformixImageFilter()
        transformixImageFilter.SetTransformParameterMap(transformParameterMap)
        transformixImageFilter.SetMovingImage(moving_image)
        transformixImageFilter.Execute()
        result_image_transformed = transformixImageFilter.GetResultImage()
        result_image_transformed = sitk.Cast(result_image_transformed, sitk.sitkUInt8)
        parts = file_name.split("_")
        transformed_image_name = f"{parts[0]}_{parts[1]}_transformed.png"
        sitk.WriteImage(result_image_transformed, os.path.join('registration/transformed_regions', transformed_image_name))

Overlay transformed regions on the image

In [ ]:
transformed_images_folder = 'registration/transformed_regions'
transformed_images = os.listdir(transformed_images_folder)
gray_folder_path = 'registration/grayscale'
overlay_folder_path = 'registration/overlayed_regions'

# Loop through each file
for file_name in transformed_images:
    file_path = os.path.join(transformed_images_folder, file_name)
    if file_name.lower().endswith(('.png')):
        parts = file_name.split("_")
        grayscale_image_path = f"{gray_folder_path}/{parts[0]}_{parts[1]}.png"

        # Read images
        gray_image = cv2.imread(grayscale_image_path)
        transformed_mask = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        mask_rgba = cv2.cvtColor(transformed_mask, cv2.COLOR_GRAY2BGRA)

        # Set alpha channel values
        alpha_factor = 1.0
        mask_rgba[:, :, 3] = transformed_mask * alpha_factor

        # Convert grayscale image to have 3 channels (RGB)
        gray_image_rgb = cv2.cvtColor(gray_image, cv2.COLOR_BGR2BGRA)

        # Overlay the binary mask region on the grayscale image
        result = cv2.addWeighted(gray_image_rgb, 1, mask_rgba, 0.5, 0)

        # Save the overlayed image with a new name
        overlay_file_name = f"{parts[0]}_{parts[1]}_overlayed_regions.png"
        overlay_file_path = os.path.join(overlay_folder_path, overlay_file_name)
        cv2.imwrite(overlay_file_path, result)

        print(f'Processed: {file_name}, overlayed image saved as: {overlay_file_name}')

Check unique intensity values on the transformed image

In [9]:
atlas = cv2.imread('registration/rgb/atlas_six_labels.png')
transformed_image = cv2.imread('registration/transformed_regions/009121_I0000831_transformed.png')

# Check image shape
print(atlas.shape)
print(transformed_image.shape)

# Convert the images to grayscale
gray_atlas = cv2.cvtColor(atlas, cv2.COLOR_BGR2GRAY)
gray_transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_BGR2GRAY)

# Flatten the images to 1D arrays
flatten_atlas = gray_atlas.flatten()
flatten_transformed_image = gray_transformed_image.flatten()

# Find unique pixel values
unique_values_atlas = np.unique(flatten_atlas)
unique_values_transformed_image = np.unique(flatten_transformed_image)

# Display unique pixel values
print("Unique pixel values in atlas:")
print(unique_values_atlas)

print("\nUnique pixel values in transformed image:")
print(unique_values_transformed_image)


(2155, 1924, 3)
(2948, 1985, 3)
Unique pixel values in atlas:
[  0  15  38  53  75  90 113]

Unique pixel values in transformed image:
[  0   9  27  36  91 100 118]
